In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa as lb
import librosa.display
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers,models,Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import tensorflow as tf
import os
import gc
from PIL import Image as PIL_Image
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2026-01-06 15:04:12.396643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767711852.579966      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767711852.629241      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767711853.055789      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767711853.055832      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767711853.055835      55 computation_placer.cc:177] computation placer alr

In [2]:
def format_shape(data, target_height=128, target_width=1000):
    data = np.asarray(data, dtype=np.float32)


#1D -> 2D
    if data.ndim == 1:
        data = data[np.newaxis, :]

    # Min-max normalization
    data=(255 * (data - np.min(data)) / (np.max(data) - np.min(data))).astype(np.uint8)

    # Resize rows to target_height
    rows, cols = data.shape
    if rows < target_height:
        reps = int(np.ceil(target_height / rows))
        data = np.tile(data, (reps, 1))[:target_height, :]
    elif rows > target_height:
        data = data[:target_height, :]

    # Resize columns to target_width
    rows, cols = data.shape
    if cols < target_width:
        pad_width = target_width - cols
        data = np.pad(data, ((0,0), (0, pad_width)), mode="constant")
    elif cols > target_width:
        data = data[:, :target_width]

    return data.astype(np.float32)

In [3]:
#this function will be responsible to convert all the audios into images
def audio_to_image(file=None,max_size=1000,y=None,sr=22050):
    #loading up the image
    if not file is None:
        if y is None:
            y,sr=librosa.load(file,sr=sr)
    y = np.asarray(y, dtype=np.float32)

    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=224, n_fft=2048, hop_length=384)

    #next channel which mfcc is which is audio graph that is audible to human
    mels_db = librosa.power_to_db(mels, ref=np.max)

    mels_pcen = librosa.pcen(mels * (2*31), sr=sr) #frequency ma compress

    mels_delta = librosa.feature.delta(mels_db) #first derivative

     # 3. Enhanced Normalization to 0-255
    def norm(X):
        X -= X.min()
        X /= (X.max() + 1e-9)
        return (X 255).astype(np.uint8)   # <--------------

    # # Stack: DB, PCEN, and Delta
    # return np.dstack([norm(mels_db), norm(mels_pcen), norm(mels_delta)])


    def normalize(X):
        x_min, x_max = X.min(), X.max()
        if x_max - x_min > 0:
            return (255 * (X - x_min) / (x_max - x_min)).astype(np.uint8)
        return np.zeros_like(X, dtype=np.uint8)

    layer0 = format_shape(mels_db)
    layer1 = format_shape(mels_delta)
    layer2 = format_shape(mels_pcen)

    #this makes cube by taking 3 images and stacking on top of each other
    final_image = np.dstack([layer0, layer1, layer2]).astype(np.float32)
    return final_image,y,sr

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2750401078.py, line 22)

In [4]:
def convert_dir_to_audio(file_path, output_dir):
    class_names = [d for d in os.listdir(file_path) if os.path.isdir(os.path.join(file_path, d))]

    for i, class_name in enumerate(class_names):
        class_input_path = os.path.join(file_path, class_name)
        save_folder = os.path.join(output_dir, class_name)
        os.makedirs(save_folder, exist_ok=True)

        print(f"--- Processing Class: {class_name} ---")

        for file_name in os.listdir(class_input_path):
            if file_name.endswith((".wav", ".m4a")):
                full_file_path = os.path.join(class_input_path, file_name)

                try:
                    # 1. Process Original
                    # Note: We use 'pixel_data' everywhere now
                    pixel_data, y, sr = audio_to_image(full_file_path, y=None)

# Save Original
                    im = PIL_Image.fromarray(pixel_data.astype(np.uint8))
                    im.save(os.path.join(save_folder, f"{file_name}_orig.png"))

# Process Augmentations
                    for j in range(3): 
                        y_aug = augment_audio(y, sr)
                        aug_pixeldata, _, _ = audio_to_image(file=None, y=y_aug, sr=sr)

                        im_aug = PIL_Image.fromarray(aug_pixel_data.astype(np.uint8))
                        # Use f-string to safely combine string and number j
                        im_aug.save(os.path.join(save_folder, f"{file_name}aug{j}.png"))

                        del y_aug, aug_pixel_data 

                    # Cleanup the original data for this file
                    del pixel_data, y

                except Exception as e:
                    print(f"Skipping {file_name}: {e}")

        gc.collect()

    return class_names

In [7]:
def augment_audio(y,sr):
    if np.random.random()>0.5:
        y=librosa.effects.pitch_shift(y,sr=sr,n_steps=np.random.uniform(-2,2))
        y = np.asarray(y, dtype=np.float32)
    y = y * np.random.uniform(0.6, 1.4) # Randomly quieter or louder
    noise_amp=0.005 * np.random.uniform()*np.amax(y)
    noise_amp = 0.005 * np.random.uniform() * np.amax(np.abs(y)) if y.size > 0 else 0.0
    y = y + noise_amp * np.random.normal(size=y.shape).astype(np.float32)
    return y.astype(np.float32)
    

In [8]:
def split_data(
    x,
    y,
    test_size=0.2,
    val_size=0.5,
    random_state=42,
    stratify=True
):
    x = np.array(x)
    y = np.array(y)

    strat = y if stratify else None

    # Train / temp split
    x_train, x_temp, y_train, y_temp = train_test_split(
        x,
        y,
        test_size=test_size,
        random_state=random_state,
        stratify=strat
    )

    # Val / test split
    x_val, x_test, y_val, y_test = train_test_split(
        x_temp,
        y_temp,
        test_size=val_size,
        random_state=random_state,
        stratify=(y_temp if stratify else None)
    )

    return x_train, y_train, x_val, y_val, x_test, y_test

In [9]:
def for_single_audio(file):
    img,y,sr=audio_to_image(file)
    img_array=np.expand_dims(img_array, axis=0)

In [7]:
input="/kaggle/input/forest-audio-dataset-added/forestdataset"
output="/kaggle/working/processed"
class_names=convert_dir_to_audio(input,output)

NameError: name 'convert_dir_to_audio' is not defined

In [ ]:
import shutil

# Zip the folder for easy download
shutil.make_archive("/kaggle/working/forest_processed", 'zip', "/kaggle/working/processed")

In [10]:
from tensorflow.keras.utils import image_dataset_from_directory, load_img, img_to_array
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

In [11]:
BATCH_SIZE=32
IMG_SIZE=(224,224)
SEED=42
EXTRACT_PATH="/kaggle/input/forest-audio-sonograph-final"
selected_class=["natural sound","unnatural"]

In [12]:
#loading up images and dividing them for the neural net
train_data=tf.keras.utils.image_dataset_from_directory(
    EXTRACT_PATH,
    class_names=selected_class,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=(128, 1000),
    batch_size=BATCH_SIZE,
    label_mode="binary",
    color_mode = 'rgb'
)

#validation dataset
validation_data=tf.keras.utils.image_dataset_from_directory(
    EXTRACT_PATH,
    class_names=selected_class,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=(128, 1000),
    batch_size=BATCH_SIZE,
    label_mode="binary",
    color_mode = 'rgb'
)

Found 12676 files belonging to 2 classes.
Using 10141 files for training.


I0000 00:00:1767712058.733890      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1767712058.737777      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 12676 files belonging to 2 classes.
Using 2535 files for validation.


In [13]:
#now dividing the validation data set into 
total_number_of_batches_in_validation_data=validation_data.cardinality().numpy() #converts total number of batches in the set and converts the tensor into python integer with .numpy function here
no_of_batches_in_validation_data=total_number_of_batches_in_validation_data//2 #this is a floor division operator


#now from the set of batches of the image creating subset into the validation and the test subset
validation=validation_data.take(no_of_batches_in_validation_data)
test=validation_data.skip(no_of_batches_in_validation_data)
print(f"Class names: {train_data.class_names}")
print("For values:\n")
for i, class_name in enumerate(train_data.class_names):
    print(f"{class_name}:{i}\n")

Class names: ['natural sound', 'unnatural']
For values:

natural sound:0

unnatural:1



In [14]:
#now we take pretrained model and train them for our data

data_augmentation = tf.keras.Sequential([
    # 1. Horizontal Shift (Time Shift) - Very important for forest sounds
    tf.keras.layers.RandomTranslation(
        height_factor=0.0, 
        width_factor=0.2, # Be aggressive with time, forest sounds can start anytime
        fill_mode='wrap'  # 'wrap' is better for audio as it loops the sound
    ),

    # 2. Small Vertical Shift (Pitch Shift)
    tf.keras.layers.RandomTranslation(
        height_factor=0.05, # Keep this small so you don't change species signatures
        width_factor=0.0,
        fill_mode='constant'
    ),

    # 3. Random Noise (Simulates Phone Mic Hiss)
    tf.keras.layers.GaussianNoise(0.02), 

    # 4. Brightness/Contrast (Simulates Different Audio Gains/Volumes)
    tf.keras.layers.RandomBrightness(factor=0.1),
    tf.keras.layers.RandomContrast(factor=0.1),
])

In [15]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import efficientnet

# 1. Define the Backbone (This is your 'mobile' or 'mobile2')
base_model = efficientnet.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False

# 2. Explicitly define the Tensor Flow
inputs = layers.Input(shape=(128, 1000, 3)) # Match your notebook's spectrograph shape

# Pass through your sequential augmentation block
x = data_augmentation(inputs) 

x = layers.Dropout(0.3)(x)

# Resize and Preprocess
x = layers.Resizing(224, 224)(x)
x = layers.Lambda(efficientnet.preprocess_input)(x)

# Pass to EfficientNet - using 'training=False' is safer for frozen layers
x = base_model(x, training=False)   #<-------------

# Top Layers
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
# x = layers.Dropout(0.5)(x)

# Choose your output based on which model you are training:
# For Model 1 (Binary): 
outputs = layers.Dense(1, activation="sigmoid")(x)
# For Model 2 (3-class: Fire, Logging, Poaching):
# outputs = layers.Dense(3, activation="softmax")(x)

# 3. Create the Model
model = models.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="adam",
              loss="binary_crossentropy", # Use 'sparse_categorical_crossentropy' for Model 2
              metrics=["accuracy"])

history=model.fit(
    train_data,
    validation_data=validation,
    epochs=50,
    batch_size=32,
    callbacks=[
        EarlyStopping(
            monitor="val_accuracy", #checks for val_accuracy
            patience=5,#wait tills 5 epochs
            restore_best_weights=True,#uses best weight
        ),
        ModelCheckpoint(
            "best_model.keras",#givesbest model according to val_accuracy
            monitor="val_accuracy",
            save_best_only=True,
            verbose=1 #prints only certain line of epoch for 1 and for 0 is silence and for 2 is every line
        )
    ]
)
test_loss,test_acc=model.evaluate(validation)
print(f"Accuracy for validation dataset: {test_acc:4f}")

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50


E0000 00:00:1767712098.664060      55 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1767712101.548996     134 cuda_dnn.cc:529] Loaded cuDNN version 91002


317/317 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.7243 - loss: 0.5449
Epoch 1: val_accuracy improved from -inf to 0.75156, saving model to best_model.keras
317/317 ━━━━━━━━━━━━━━━━━━━━ 44s 99ms/step - accuracy: 0.7244 - loss: 0.5448 - val_accuracy: 0.7516 - val_loss: 0.5143
Epoch 2/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.7772 - loss: 0.4685
Epoch 2: val_accuracy did not improve from 0.75156
317/317 ━━━━━━━━━━━━━━━━━━━━ 28s 88ms/step - accuracy: 0.7773 - loss: 0.4684 - val_accuracy: 0.7063 - val_loss: 0.5185
Epoch 3/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.7881 - loss: 0.4411
Epoch 3: val_accuracy did not improve from 0.75156
317/317 ━━━━━━━━━━━━━━━━━━━━ 28s 89ms/step - accuracy: 0.7881 - loss: 0.4411 - val_accuracy: 0.7219 - val_loss: 0.5389
Epoch 4/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.7906 - loss: 0.4239
Epoch 4: val_accuracy did not improve from 0.75156
317/317 ━━━━━━━━━━━━━━━━━━━━ 29s 91ms/step - accuracy: 0.7906 - los

In [17]:
result=model.evaluate(test)
print(f"For the unseen test data of the entire training this model has accuracy of {result[1]:.4f}")

40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - accuracy: 0.7728 - loss: 0.4869
For the unseen test data of the entire training this model has accuracy of 0.7721


In [18]:
os.makedirs("/kaggle/working/models",exist_ok=True)
model.save("/kaggle/working/models/audio_forest.keras")
import shutil

# Zip the folder for easy download
shutil.make_archive("/kaggle/working/models", 'zip', "/kaggle/working/models")

'/kaggle/working/models.zip'

In [19]:
selected_class=["fire","logging", "poaching"]

In [20]:
#loading up images and dividing them for the neural net
train_data=tf.keras.utils.image_dataset_from_directory(
    EXTRACT_PATH,
    image_size=(128, 1000),
    class_names=selected_class,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    batch_size=BATCH_SIZE,
    label_mode="int" ,
    color_mode='rgb'
)

#validation dataset
validation_data=tf.keras.utils.image_dataset_from_directory(
    EXTRACT_PATH,
    image_size=(128, 1000),
    class_names=selected_class,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    batch_size=BATCH_SIZE,
    label_mode="int",
    color_mode='rgb'
)


Found 5768 files belonging to 3 classes.
Using 4615 files for training.
Found 5768 files belonging to 3 classes.
Using 1153 files for validation.


In [21]:
#now dividing the validation data set into 
total_number_of_batches_in_validation_data=validation_data.cardinality().numpy() #converts total number of batches in the set and converts the tensor into python integer with .numpy function here
no_of_batches_in_validation_data=total_number_of_batches_in_validation_data//2 #this is a floor division operator


#now from the set of batches of the image creating subset into the validation and the test subset
validation=validation_data.take(no_of_batches_in_validation_data)
test=validation_data.skip(no_of_batches_in_validation_data)


In [22]:
print(f"Class names: {train_data.class_names}")
print("For values:\n")
for i, class_name in enumerate(train_data.class_names):
    print(f"{class_name}:{i}\n")


Class names: ['fire', 'logging', 'poaching']
For values:

fire:0

logging:1

poaching:2



In [23]:

def multi_class_model():
    # 1. Define Input
    inputs = tf.keras.Input(shape=(128, 1000, 3))
    x = data_augmentation(inputs)
    x = layers.Resizing(224, 224)(x)


    base_model2 = tf.keras.applications.EfficientNetB0(
        input_shape=(224, 224, 3),
        include_top=False,
        weights="imagenet"
    )
    base_model2.trainable = False
    
    x = base_model2(x, training=False) # training=False keeps BatchNormalization in inference mode

    # 5. The "Head"
    x = layers.GlobalMaxPooling2D()(x)
    x = layers.Dense(256, activation="relu")(x)
    outputs = layers.Dense(3, activation="softmax")(x)

    # 6. Create the Model
    model2 = models.Model(inputs=inputs, outputs=outputs)
    
    return model2
model2 = multi_class_model()

model2.compile(optimizer="adam",
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])

history=model2.fit(
    train_data,
    validation_data=validation,
    epochs=50,
    batch_size=32,
    callbacks=[
        EarlyStopping(
            monitor="val_accuracy", #checks for val_accuracy
            patience=5,#wait tills 5 epochs
            restore_best_weights=True,#uses best weight
        ),
        ModelCheckpoint(
            "best_model.keras",#givesbest model according to val_accuracy
            monitor="val_accuracy",
            save_best_only=True,
            verbose=1 #prints only certain line of epoch for 1 and for 0 is silence and for 2 is every line
        )
    ]
)

test_loss,test_acc=model2.evaluate(validation)
print(f"Accuracy for validation dataset: {test_acc:4f}")

Epoch 1/50


E0000 00:00:1767712631.787478      55 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_2_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


144/145 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.7486 - loss: 1.1996
Epoch 1: val_accuracy improved from -inf to 0.90451, saving model to best_model.keras
145/145 ━━━━━━━━━━━━━━━━━━━━ 26s 113ms/step - accuracy: 0.7494 - loss: 1.1921 - val_accuracy: 0.9045 - val_loss: 0.2311
Epoch 2/50
144/145 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.8489 - loss: 0.3604
Epoch 2: val_accuracy improved from 0.90451 to 0.90972, saving model to best_model.keras
145/145 ━━━━━━━━━━━━━━━━━━━━ 14s 94ms/step - accuracy: 0.8490 - loss: 0.3601 - val_accuracy: 0.9097 - val_loss: 0.1991
Epoch 3/50
144/145 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.8839 - loss: 0.2882
Epoch 3: val_accuracy did not improve from 0.90972
145/145 ━━━━━━━━━━━━━━━━━━━━ 13s 90ms/step - accuracy: 0.8840 - loss: 0.2881 - val_accuracy: 0.8958 - val_loss: 0.2696
Epoch 4/50
144/145 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.8897 - loss: 0.2714
Epoch 4: val_accuracy improved from 0.90972 to 0.91493, saving model to best_mo

In [24]:
result=model2.evaluate(test)
print(f"For the unseen test data of the entire training this model has accuracy of {result[1]:.4f}\\")

19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.9443 - loss: 0.1713
For the unseen test data of the entire training this model has accuracy of 0.9411\


In [25]:
result=model2.evaluate(train_data)
print(f"For the unseen test data of the entire training this model has accuracy of {result[1]:.4f}\\")

145/145 ━━━━━━━━━━━━━━━━━━━━ 10s 70ms/step - accuracy: 0.9321 - loss: 0.1869
For the unseen test data of the entire training this model has accuracy of 0.9333\


In [ ]:

# ===== ADD THIS CELL AFTER model.fit() COMPLETES =====

print("\n" + "="*80)
print("🔍 VERIFYING MODEL BEFORE SAVING")
print("="*80)

# Check the model we just trained
print(f"Model input shape: {model2.input_shape}")
print(f"Expected:          (None, 128, 1000, 3)")

# Test with a sample batch
for images, labels in train_data.take(1):
    print(f"\nSample batch shape: {images.shape}")
    print(f"Sample batch channels: {images.shape[-1]}")
    
    # Try prediction
    try:
        pred = model.predict(images[:1])
        print(f"✅ Model can predict on training data")
        print(f"   Prediction shape: {pred.shape}")
    except Exception as e:
        print(f"❌ Model CANNOT predict: {e}")
        print("   TRAINING DATA AND MODEL DON'T MATCH!")

if model2.input_shape == (None, 128, 1000, 3):
    print("\n✅ MODEL ARCHITECTURE IS CORRECT")
else:
    print(f"\n❌ MODEL ARCHITECTURE IS WRONG!")
    print(f"   Model expects: {model2.input_shape}")
    print(f"   But should be: (None, 128, 1000, 3)")
    raise ValueError("Model has wrong input shape! Fix training code!")

print("="*80 + "\n")

# Now save
os.makedirs("/kaggle/working/models", exist_ok=True)
model2.save("/kaggle/working/models/audio_forest_69.keras")

# Verify the SAVED file
print("\n" + "="*80)
print("🔍 VERIFYING SAVED FILE")
print("="*80)

test_model = tf.keras.models.load_model(
    "/kaggle/working/models/audio_forest_69.keras", 
    compile=False
)

print(f"Saved model input shape: {test_model.input_shape}")

if test_model.input_shape == (None, 128, 1000, 3):
    print("✅ SAVED MODEL IS CORRECT - Safe to download!")
else:
    print("❌ SAVED MODEL IS WRONG!")
    raise ValueError("Something went wrong during save!")
    
print("="*80 + "\n")

# Zip it
import shutil
shutil.make_archive("/kaggle/working/models", 'zip', "/kaggle/working/models")
print("✅ models.zip created - MODEL IS VERIFIED CORRECT!")




In [ ]:
os.makedirs("/kaggle/working/models",exist_ok=True)
model.save("/kaggle/working/models/audio_multi_classification.keras")
import shutil

# Zip the folder for easy download
shutil.make_archive("/kaggle/working/maheshdalle", 'zip', "/kaggle/working/models")
